In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
#!pip install sentence_transformers
#!pip install konlpy

In [3]:
장소_리스트 = ['부산민주공원','민락수변공원','흰여울문화마을','해운대','해동용궁사','태종대','감천문화마을']

In [4]:
def 키버트_키워드_추출(장소,최소그램,최대그램,추출키워드개수,상위키워드개수):
    평점 = []
    키워드_총_리스트=[]
    s_w=set(['곳','수','것','추천','정도','사람','가슴','때','정말','볼','꼭',
             '좀','날','그','번','조금','가면','시간','타고','다누','방문',
             '때문','다시','예전','보고','중간','주변','한번','변','근처',
             '중앙','근처','센터','정말','정도','워낙','마치','아주','대박',
             '더욱','단지','워낙','이기','다만','인득','정말','자주','완전','금방'])
    for k in tqdm(장소):
        t_d = pd.read_csv(f'{k}.csv')
        doc=""
        for m in t_d['리뷰']:
            doc += m
        doc = doc.replace(k,'')
        okt = Okt()
        tokenized_doc = okt.nouns(doc)
        end_d=[w for w in tokenized_doc if not w in s_w]
        tokenized_nouns = ' '.join(end_d)
        n_gram_range = (최소그램, 최대그램)
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()
        model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)
        candidates_idx = list(distances.argsort()[0][-상위키워드개수:])
        candidates_vals = [candidates[index] for index in candidates_idx]
        distances_candidates = distances_candidates[np.ix_(candidates_idx, candidates_idx)]
        min_sim = np.inf
        candidate = None
        for combination in itertools.combinations(range(len(candidates_idx)), 추출키워드개수):
            sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
            if sim < min_sim:
                candidate = combination
                min_sim = sim
        키워드_리스트=[]
        for idx in candidate:
            키워드_리스트.append(candidates_vals[idx])
        키워드_총_리스트.append(키워드_리스트)        
        평점.append(t_d['평점'].mean())
        print(f'''장소 : {k}
키워드 : {키워드_리스트}
평점 : {t_d['평점'].mean()}        
        ''')
    장소_키워드_평점=pd.DataFrame(list(zip(장소, 키워드_총_리스트, 평점)) ,columns=['장소','키워드','평점'])
    return 장소_키워드_평점

In [5]:
%%time
결과 = 키버트_키워드_추출(장소=장소_리스트,최소그램=2,최대그램=3,추출키워드개수=5,상위키워드개수=20)
결과

 14%|█▍        | 1/7 [00:21<02:09, 21.64s/it]

장소 : 부산민주공원
키워드 : ['벚꽃 구경', '시내버스 이동', '왕겹 벚꽃', '도서관 왕겹', '민주 공원 시내버스']
평점 : 4.6875        
        


 29%|██▊       | 2/7 [00:36<01:26, 17.40s/it]

장소 : 민락수변공원
키워드 : ['판매 돗자리 여름', '재미 접근', '광안대교 야경 식사', '바다 낭만', '광안리해수욕장 모래 해변']
평점 : 4.104166666666667        
        


 43%|████▎     | 3/7 [00:51<01:05, 16.47s/it]

...

 86%|████████▌ | 6/7 [01:34<00:14, 14.99s/it]

장소 : 태종대
키워드 : ['관광명소 꼬마 열차', '접근성 대중교통 이용', '전망 시원 재미', '관광열차 절벽', '망망대해 편이 해안가']
평점 : 4.36        
        


100%|██████████| 7/7 [01:49<00:00, 15.59s/it]

장소 : 감천문화마을
키워드 : ['마을 음식 간식', '구경 재미 골목길', '관광 질서 관광객', '공짜 가족 여행', '질서 관광객 주말']
평점 : 4.15        
        
CPU times: user 1min 6s, sys: 9.68 s, total: 1min 16s
Wall time: 1min 49s


,장소,키워드,평점
0,부산민주공원,"[벚꽃 구경, 시내버스 이동, 왕겹 벚꽃, 도서관 왕겹, 민주 공원 시내버스]",4.687500
1,민락수변공원,"[판매 돗자리 여름, 재미 접근, 광안대교 야경 식사, 바다 낭만, 광안리해수욕장 ...",4.104167
2,흰여울문화마을,"[한복 사진 거리음식, 정돈 관광, 한복 상가 상업, 분위기 간식 한복, 길거리 간...",3.630000
3,해운대,"[북적 인파 매력, 여름 해수욕 야경, 비치 이자 관광지, 도심 해수욕장 인기, 해...",4.450000
4,해동용궁사,"[관광객 고요한 사색, 바다 웅장, 금색 불상 바닷가, 관광객 독특 경험, 평일 휴...",4.240000
5,태종대,"[관광명소 꼬마 열차, 접근성 대중교통 이용, 전망 시원 재미, 관광열차 절벽, 망...",4.360000
6,감천문화마을,"[마을 음식 간식, 구경 재미 골목길, 관광 질서 관광객, 공짜 가족 여행, 질서 ...",4.150000


In [6]:
결과

,장소,키워드,평점
0,부산민주공원,"[벚꽃 구경, 시내버스 이동, 왕겹 벚꽃, 도서관 왕겹, 민주 공원 시내버스]",4.687500
1,민락수변공원,"[판매 돗자리 여름, 재미 접근, 광안대교 야경 식사, 바다 낭만, 광안리해수욕장 ...",4.104167
2,흰여울문화마을,"[한복 사진 거리음식, 정돈 관광, 한복 상가 상업, 분위기 간식 한복, 길거리 간...",3.630000
3,해운대,"[북적 인파 매력, 여름 해수욕 야경, 비치 이자 관광지, 도심 해수욕장 인기, 해...",4.450000
4,해동용궁사,"[관광객 고요한 사색, 바다 웅장, 금색 불상 바닷가, 관광객 독특 경험, 평일 휴...",4.240000
5,태종대,"[관광명소 꼬마 열차, 접근성 대중교통 이용, 전망 시원 재미, 관광열차 절벽, 망...",4.360000
6,감천문화마을,"[마을 음식 간식, 구경 재미 골목길, 관광 질서 관광객, 공짜 가족 여행, 질서 ...",4.150000
